In [1]:
import numpy as np
from scipy.linalg import expm

# Define Pauli matrices
I = np.eye(2, dtype=complex)
X = np.array([[0, 1], [1, 0]], dtype=complex)
Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
Z = np.array([[1, 0], [0, -1]], dtype=complex)

# Tensor product helper
def kron(*ops):
    """Computes the Kronecker product of multiple matrices."""
    result = ops[0]
    for op in ops[1:]:
        result = np.kron(result, op)
    return result

# Define two-site fermionic Hamiltonian using Jordan-Wigner transformation
t = 1.0  # Hopping amplitude
H = -t * ( 
    kron(X, X) + kron(Y, Y)  # c1† c2 + c2† c1 in spin representation
)

# Compute unitary time evolution operator U = exp(-i H t)
t_evolution = 1.0  # Evolution time
U = expm(-1j * H * t_evolution)

# Print results
print("Hamiltonian H:\n", np.round(H, 3))
print("\nTime evolution operator U:\n", np.round(U, 3))

# Apply U to an initial state |01⟩ (fermion at site 2)
psi_0 = np.array([0, 1, 0, 0], dtype=complex)  # |01⟩ state
psi_t = U @ psi_0  # Time evolution

print("\nEvolved state ψ(t):\n", np.round(psi_t, 3))


Hamiltonian H:
 [[-0.+0.j -0.+0.j -0.+0.j -0.+0.j]
 [-0.+0.j -0.+0.j -2.+0.j -0.+0.j]
 [-0.+0.j -2.+0.j -0.+0.j -0.+0.j]
 [-0.+0.j -0.+0.j -0.+0.j -0.+0.j]]

Time evolution operator U:
 [[ 1.   +0.j     0.   +0.j     0.   +0.j     0.   +0.j   ]
 [ 0.   +0.j    -0.416+0.j     0.   +0.909j  0.   +0.j   ]
 [ 0.   +0.j     0.   +0.909j -0.416+0.j     0.   +0.j   ]
 [ 0.   +0.j     0.   +0.j     0.   +0.j     1.   +0.j   ]]

Evolved state ψ(t):
 [ 0.   +0.j    -0.416+0.j     0.   +0.909j  0.   +0.j   ]


In [2]:
import numpy as np
import re

# Define Pauli matrices
I = np.eye(2, dtype=complex)
X = np.array([[0, 1], [1, 0]], dtype=complex)
Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
Z = np.array([[1, 0], [0, -1]], dtype=complex)

# Tensor product helper function
def kron(*ops):
    """Computes the Kronecker product of multiple matrices."""
    result = ops[0]
    for op in ops[1:]:
        result = np.kron(result, op)
    return result

# Jordan-Wigner transformation function
def jordan_wigner_translate(term, t=1.0, n_sites=2):
    """
    Translates a fermionic Hamiltonian term (e.g., 'c1† c2 + c2† c1') 
    into spin representation using the Jordan-Wigner transformation.

    Parameters:
        term (str): Fermionic operator string (e.g., 'c1† c2 + c2† c1').
        t (float): Coefficient (default 1.0).
        n_sites (int): Number of qubits (default 2).

    Returns:
        np.ndarray: Spin Hamiltonian matrix.
    """
    # Initialize empty Hamiltonian matrix
    H_spin = np.zeros((2**n_sites, 2**n_sites), dtype=complex)

    # Parse terms from input string (supports + and - operations)
    terms = term.split('+')
    parsed_terms = []
    
    for t in terms:
        t = t.strip()
        if '-' in t[1:]:  # Handles cases like 'c1† c2 - c2† c1'
            sub_terms = t.split('-')
            parsed_terms.append(sub_terms[0].strip())
            parsed_terms.append('-' + sub_terms[1].strip())  # Keep negative sign
        else:
            parsed_terms.append(t)

    # Define JW transformation for c† and c
    def jw_transform(site, dagger=False):
        """
        Jordan-Wigner transformation for c_j or c_j†.
        """
        I_list = [I] * n_sites
        if dagger:
            op = (X - 1j * Y) / 2  # c† = (X - iY) / 2
        else:
            op = (X + 1j * Y) / 2  # c = (X + iY) / 2

        # Apply Jordan-Wigner string (Z chain for sites before this one)
        for i in range(site):
            I_list[i] = Z

        # Apply the operator at the target site
        I_list[site] = op

        return kron(*I_list)

    # Process each term
    for t in parsed_terms:
        factor = -t if t.startswith('-') else 1  # Handle minus sign
        t = t.lstrip('-')  # Remove the minus sign

        # Extract operator indices and whether they are creation/annihilation
        matches = re.findall(r'c(\d+)(†?)', t)
        if len(matches) != 2:
            raise ValueError(f"Invalid fermionic operator format: {t}")

        # Extract indices (convert to 0-based)
        site1 = int(matches[0][0]) - 1
        site2 = int(matches[1][0]) - 1

        # Determine if they are creation or annihilation operators
        op1 = jw_transform(site1, dagger=(matches[0][1] == '†'))
        op2 = jw_transform(site2, dagger=(matches[1][1] == '†'))

        # Compute the full term contribution
        H_spin += factor * t * (op1 @ op2)  # Matrix multiplication

    return H_spin

# Example usage
fermionic_term = "c1† c2 + c2† c1"
H_translated = jordan_wigner_translate(fermionic_term, t=1.0, n_sites=2)

# Print results
print("Hamiltonian in spin representation (Jordan-Wigner transformed):")
print(np.round(H_translated, 3))


UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('<U6'), dtype('complex128')) -> None